# Урок 6. Задача look-alike
**Домашнее задание**

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать feature engineering
3. обучить любой классификатор (какой вам нравится)
4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
5. применить random negative sampling для построения классификатора в новых условиях
6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

<b>Бонусный вопрос:</b>

Как вы думаете, какой из методов на практике является более предпочтительным: `random negative sampling` или `2-step approach`?

Ссылки:
1. https://arxiv.org/pdf/1811.04820.pdf
2. https://habr.com/ru/company/JetBrains-education/blog/512032/
3. https://en.wikipedia.org/wiki/Bootstrap_aggregating
4. https://www.cs.uic.edu/~liub/publications/EMNLP-2010-no-negative.pdf

## Выполнение

### 1. взять любой набор данных для бинарной классификации 
(можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

---
Взял этот датасет по возрасту https://archive.ics.uci.edu/ml/datasets/Adult

Информация о наборе данных: Извлечение было сделано Барри Беккером из базы данных переписи 1994 года. Набор достаточно чистых записей был извлечен с использованием следующих условий: ((AAGE> 16) && (AGI> 100) && (AFNLWGT> 1) && (HRSWK> 0))

Задача прогноза - определить, зарабатывает ли человек более 50 тысяч в год.

In [76]:
import re
import numpy as np
import pandas as pd

In [77]:
# from sklearn.model_selection import cross_val_score, train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix

In [78]:
df = pd.read_csv(
    'data/adult/adult.data', sep=',', index_col=False,
    names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship',
           'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'])

In [79]:
df.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


### 2. сделать feature engineering

In [80]:
# Проврка пропусков
pd.DataFrame([df.isna().sum(), df.isnull().sum()], index={'na', 'null'})

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
na,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [81]:
# Изменение целевой переменной
df['income'] = df['income'].map({' <=50K': 0, ' >50K': 1})

In [82]:
from sklearn.model_selection import train_test_split

X = pd.get_dummies(df.iloc[:, :-1])
y = df['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

### 3. обучить любой классификатор (какой вам нравится)

In [83]:
import xgboost as xgb

model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

/home/pavel/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:07:41] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [84]:
def evaluate_results(y_test, y_predict):
    """ Проверка качества модели с помощью DS-метрик"""
    from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score

    f1 = f1_score(y_test, y_predict)
    roc = roc_auc_score(y_test, y_predict)
    precision = precision_score(y_test, y_predict, average='binary')
    recall = recall_score(y_test, y_predict, average='binary')
#     print(f'Classification results: \nf1: {round(f1, 3)}\n roc: {round(roc, 3)}\n precision: {round(precision, 3)}\n recall: {round(recall, 3)}')
    return f1, precision, recall, roc

In [85]:
evaluate_results(y_test, y_predict)

(0.7090464547677262,
 0.7801691006917756,
 0.6498079385403329,
 0.7960209153416673)

### 4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

Представим, что нам неизвестны негативы и часть позитивов

In [86]:
mod_data = pd.get_dummies(df.copy())
mod_data['income'].value_counts()

0    24720
1     7841
Name: income, dtype: int64

In [87]:
# get the indices of the positives samples
pos_ind = np.where(mod_data.loc[:, 'income'].values == 1)[0]

# shuffle them
np.random.shuffle(pos_ind)

# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 1961/7841 as positives and unlabeling the rest


In [88]:
# Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)
mod_data['class_test'] = -1
mod_data.loc[pos_sample, 'class_test'] = 1
print('target variable:\n', mod_data.iloc[:, -1].value_counts())

target variable:
 -1    30600
 1     1961
Name: class_test, dtype: int64


In [89]:
x_data = mod_data.drop(['income', 'class_test'], axis=1).values  # just the X
y_labeled = mod_data.loc[:, 'class_test'].values  # new class (just the P & U)
y_positive = mod_data.loc[:, 'income'].values  # original class

### 5. применить `random negative sampling` для построения классификатора в новых условиях

In [90]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test'] == -1][:len(mod_data[mod_data['class_test'] == 1])]
sample_test = mod_data[mod_data['class_test'] == -1][len(mod_data[mod_data['class_test'] == 1]):]
pos_sample = mod_data[mod_data['class_test'] == 1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(1961, 110) (1961, 110)


In [91]:
model_rns = xgb.XGBClassifier()
model.fit(sample_train.drop(['income', 'class_test'], axis=1).values,
          sample_train.loc[:, 'income'].values)
y_predict_rns = model.predict(sample_test.drop(['income', 'class_test'], axis=1).values)

evaluate_results(sample_test.loc[:, 'income'].values, y_predict_rns)

[09:07:47] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/pavel/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


(0.6184443881428934,
 0.47421078193297717,
 0.8887675223011106,
 0.8274521098933185)

0.1 (0.6458047264580471, 0.7013027782137812, 0.5984462898473077, 0.7587768112360874)
0.25 (0.6787840802256346, 0.740766073871409, 0.6263736263736264, 0.7782929219194928)
0.5 (0.5424854194706146, 0.38821112109926803, 0.9002382370458606, 0.8386074851724317)


### 6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)

In [92]:
pd.DataFrame([
    evaluate_results(y_test, y_predict),
    evaluate_results(sample_test.loc[:, 'income'].values, y_predict_rns)
], columns={'f1', 'precision', 'recall', 'roc'}, index={'Simple xgboost', 'random negative sampling'}).round(2)

,precision,f1,roc,recall
random negative sampling,0.71,0.78,0.65,0.80
Simple xgboost,0.62,0.47,0.89,0.83


### 7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

Зависимость метрик от P не линейна. Так, при малом P, или, скорее, малом количестве размеченных данных, качество резко падает - модели не хватает примеров, чтобы нормально обучиться. Для не слишком малых P, при увеличении доли размеченных данных  precision снижается, а recall растет. Что тоже не удивительно, ведь чем больше Р тем меньше 1-Р (доля объектов 1 класса попавшая в неразмеченные данные). Чем их меньше, тем легче найти все, тем сложнее обеспечить чистоту предсказаний. 

В целом, вывод относится только к данному датасету. Объем объектов первого класса в нем примерно 7800, что составляет порядка 1/4 общего объема данных. Такой дисбаланс усугубляет ситуацию, когда изменение Р существенно влияет на структуру неразмеченных данных, а малое количетво объектов вообще - еще и на качество обучения модели. Опять же, модели толком не настраивались, гиперпараметры не подбирались.

Если же говорить о задаче look-alike вообще, то чем больше у нас размеченных данных, тем лучше. Если их достаточно много, возможно, стоит провести анализ, разбить на какие-то группы/кластеры, и решать look-alike для каждого из них. Например, поделили размеченные данные на тех, кто заплатил два и более раз (постоянные), тех, кто заплатил много (крупные) и остальных клиентов. Если "насобирается" достаточно данных по "интересным" категориям клиентов, возможно стоит сосредоточиться на их поиске. Все зависит от конкретной задачи, конкретных данных в конечном итоге. Пробовать, тестировать, делать выводы.